<a href="https://colab.research.google.com/github/cr21/Unsupervised-Machine-Learning-Clustering/blob/main/FeatureSelectionChi2Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Dataset/

/content/drive/MyDrive/Dataset


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection  import mutual_info_classif
from sklearn import linear_model
from sklearn.feature_selection import SelectFromModel

# Feature Selection using Chi2 Test

In [2]:
# Load 20 News Group dataset and split train and test data
def load20NG():
  dataset = fetch_20newsgroups(subset='all', remove=('headers','footers','quotes'), random_state= 12)
  data = dataset.data
  labels = dataset.target

  vectorizer = TfidfVectorizer(stop_words='english',min_df = 5)

  # # vectorize train data
  ngVectorData = vectorizer.fit_transform(data)
  featureNames = vectorizer.get_feature_names()
  return data, labels, ngVectorData, featureNames
data, labels, ng20Data, ng20featureNames = load20NG()
X_train, X_test, y_train, y_test = train_test_split(ng20Data, labels, test_size=0.33, random_state=42)

In [3]:
Transformed_200_X_train = SelectKBest(chi2, k=200).fit_transform(X_train, y_train)

In [4]:
Transformed_200_X_test = SelectKBest(chi2, k =200).fit_transform(X_test,y_test)

In [5]:
ngClassfier = LogisticRegression(penalty='l2', tol =0.0001, C = 1.0,n_jobs=-1)
ngClassfier.fit(Transformed_200_X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
# training error
print(f" training Error : {ngClassfier.score(Transformed_200_X_train, y_train)}")

 training Error : 0.5355615396800253


In [7]:
#testing error
print(f" testing Error : {ngClassfier.score(Transformed_200_X_test, y_test)}")

 testing Error : 0.09212218649517685


# Mutual Information for Feature Selection

In [8]:
feature_scores = mutual_info_classif(X_train, y_train, random_state=0)

In [9]:
Transformed_200_X_train = SelectKBest(mutual_info_classif, k=200).fit_transform(X_train, y_train)


In [10]:
Transformed_200_X_test = SelectKBest(mutual_info_classif, k =200).fit_transform(X_test,y_test)

In [11]:
ngClassfier = LogisticRegression(penalty='l2', tol =0.0001, C = 1.0,n_jobs=-1)
ngClassfier.fit(Transformed_200_X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
# training error
print(f" training Error : {ngClassfier.score(Transformed_200_X_train, y_train)}")

 training Error : 0.3446063678124505


In [13]:
# training error
print(f" training Error : {ngClassfier.score(Transformed_200_X_test, y_test)}")

 training Error : 0.13167202572347267


In [14]:
# Top 200 features
sorted_ngFeatureList = np.argsort(feature_scores)
reverse_sorted_ngFeatureList = sorted_ngFeatureList[::-1]
top200 = reverse_sorted_ngFeatureList[:200]
for id, featureId in enumerate(top200):
  featureNameList = np.array(ng20featureNames)[featureId]
  print(f" id : {id} , feature name : {featureNameList}")


 id : 0 , feature name : like
 id : 1 , feature name : just
 id : 2 , feature name : don
 id : 3 , feature name : know
 id : 4 , feature name : think
 id : 5 , feature name : people
 id : 6 , feature name : does
 id : 7 , feature name : time
 id : 8 , feature name : use
 id : 9 , feature name : good
 id : 10 , feature name : way
 id : 11 , feature name : make
 id : 12 , feature name : new
 id : 13 , feature name : ve
 id : 14 , feature name : used
 id : 15 , feature name : want
 id : 16 , feature name : say
 id : 17 , feature name : right
 id : 18 , feature name : need
 id : 19 , feature name : did
 id : 20 , feature name : thanks
 id : 21 , feature name : really
 id : 22 , feature name : work
 id : 23 , feature name : problem
 id : 24 , feature name : edu
 id : 25 , feature name : using
 id : 26 , feature name : believe
 id : 27 , feature name : point
 id : 28 , feature name : sure
 id : 29 , feature name : things
 id : 30 , feature name : years
 id : 31 , feature name : help
 id : 32

In [15]:
top200XTrain = X_train[:,top200 ]
top200XTest = X_test[:, top200]

In [16]:
ngClassfier = LogisticRegression(penalty='l2', tol =0.0001, C = 1.0,n_jobs=-1)
ngClassfier.fit(top200XTrain, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
# training error
print(f" training Error : {ngClassfier.score(top200XTrain, y_train)}")

 training Error : 0.3446063678124505


In [18]:
#testing error
print(f" testing Error : {ngClassfier.score(top200XTest, y_test)}")

 testing Error : 0.30884244372990355


# L1 feature selection on text

In [29]:
# scaler = StandardScaler()
# scaler.fit(X_train)
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1',solver='saga',max_iter=200),max_features=200)
sel_.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=200, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='saga',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=200, norm_order=1, prefit=False, threshold=None)

In [45]:
X_Train_200  = X_train[:, sel_.get_support()]
X_Test_200  = X_test[:, sel_.get_support()]

In [46]:
ng20Classfier_top200 = LogisticRegression(penalty='l2', tol =0.001, C = 1.0,n_jobs=-1, solver='saga',max_iter = 100)
ng20Classfier_top200.fit(X_Train_200, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='saga', tol=0.001, verbose=0,
                   warm_start=False)

In [47]:
# training error
print(f" training Error : {ng20Classfier_top200.score(X_Train_200, y_train)}")

 training Error : 0.5583716141295739


In [48]:
# training error
print(f" testing Error : {ng20Classfier_top200.score(X_Test_200, y_test)}")

 testing Error : 0.5303858520900322
